In [83]:
import os
import json
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate


with open("keys.json", "r") as f:
    keys = json.load(f)
    os.environ['OPENAI_API_KEY'] = keys['openai']
    os.environ['COHERE_API_KEY'] = keys['cohere']

llm = OpenAI()
chat_model = ChatOpenAI()


In [2]:
text = "What would be a good company name for a company that makes colorful socks?"

llm.predict(text)

'\n\nBrightfoot Socks.'

In [8]:
text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content = text)]

chat_model.predict_messages(messages, temperature = 1.9)

AIMessage(content='ColorThread Collections')

In [14]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="computers")


'What is a good name for a company that makes computers?'

In [19]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_model.predict_messages(chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming"))


AIMessage(content="J'adore programmer")

In [23]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")
    

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

chain.invoke({"text": "colors"})


['red', 'blue', 'yellow', 'green', 'orange']

# Learning expression language

In [40]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}", temperature = 0.2)
chat_model = ChatOpenAI(temperature= 0)

chain = prompt | chat_model

chain.input_schema.schema()
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/HumanMessageChunk'},
  {'$ref': '#/definitions/AIMessageChunk'},
  {'$ref': '#/definitions/ChatMessageChunk'},
  {'$ref': '#/definitions/FunctionMessageChunk'},
  {'$ref': '#/definitions/SystemMessageChunk'}],
 'definitions': {'HumanMessageChunk': {'title': 'HumanMessageChunk',
   'description': 'A Human Message chunk.',
   'type': 'object',
   'properties': {'content': {'title': 'Content', 'type': 'string'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'human',
     'enum': ['human'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'},
    'is_chunk': {'title': 'Is Chunk',
     'default': True,
     'enum': [True],
     'type': 'boolean'}},
   'required': ['content']},
  'AIMessageChunk': {'title': 'AIMessageChunk',
   'description': 'A Message chunk from an AI.',
   'type': 'object',
   'properties':

In [41]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Once upon a time, in a dense and enchanting forest, there lived a family of bears. The family consisted of a wise and gentle father bear named Benjamin, a loving and nurturing mother bear named Amelia, and their curious and playful cub named Oliver.

The bears lived harmoniously in their cozy den, surrounded by towering trees and a babbling brook. Benjamin, being the protector of the family, would venture out into the forest each day to gather food and ensure their safety. Amelia would spend her days teaching Oliver about the wonders of nature, sharing stories of their ancestors, and instilling in him the values of kindness and respect.

One sunny morning, as Benjamin was exploring a new part of the forest, he stumbled upon a lost and frightened baby deer. The poor creature had strayed far from its family and was unable to find its way back. Benjamin's compassionate heart couldn't bear to see the little deer in distress, so he gently picked it up with his strong paws and carried it bac

In [31]:
chain.batch([{"topic": "dogs"}, {"topic": "cats"}])

[AIMessage(content="Why don't dogs make good dancers?\n\nBecause they have two left feet-paws!"),
 AIMessage(content='Sure, here\'s a cat joke for you:\n\nWhy don\'t cats play poker in the wild?\n\nBecause they prefer "cheetahs"!')]

In [45]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(['harrison worked at kensho'], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {"context": retriever.with_config(run_name = 'Bla'), "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log("where did harrison work?", include_names=['Docs', 'Bla'], diff=False):
    print(chunk)



RunLog({'final_output': None,
 'id': '1bfd0621-2d8d-4083-a83f-1d9a2ce9a2e6',
 'logs': {},
 'streamed_output': []})
RunLog({'final_output': None,
 'id': '1bfd0621-2d8d-4083-a83f-1d9a2ce9a2e6',
 'logs': {'Bla': {'end_time': None,
                  'final_output': None,
                  'id': '215909e2-b81f-4833-a703-328e42626ad3',
                  'metadata': {},
                  'name': 'Bla',
                  'start_time': '2023-10-13T16:30:17.903',
                  'streamed_output_str': [],
                  'tags': ['map:key:context', 'FAISS'],
                  'type': 'retriever'}},
 'streamed_output': []})
RunLog({'final_output': None,
 'id': '1bfd0621-2d8d-4083-a83f-1d9a2ce9a2e6',
 'logs': {'Bla': {'end_time': '2023-10-13T16:30:20.898',
                  'final_output': {'documents': [Document(page_content='harrison worked at kensho')]},
                  'id': '215909e2-b81f-4833-a703-328e42626ad3',
                  'metadata': {},
                  'name': 'Bla',
       

In [100]:
from langchain.schema.runnable import RunnableParallel
chat_model = ChatOpenAI(temperature=1.6)

chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | chat_model
chain2 = ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}") | chat_model
combined =RunnableParallel(joke=chain1, poem=chain2)

In [104]:
chain1.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear socks?\nBecause they already have bare paws!")

In [105]:
chain2.invoke({"topic": "bears"})

AIMessage(content="In wild wood's lair,                      Rough paws with clumsy flair,\nMajestic bears roam.                 Mighty silence carved on their tome.")

In [106]:
combined.invoke({"topic": "bears"})

{'joke': AIMessage(content='Why don\'t bears use cellphones?\n \nBecause they always break the "bear" zones!'),
 'poem': AIMessage(content='Majestic and wild, fierce and friendly,\nBears in the forest, a love ever so pure.')}

In [52]:
from langchain.schema.runnable import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n"),
        ("human", "{equation_statement}")
    ]
)

model = ChatOpenAI(temperature=0)

runnable = {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION:
Subtracting 7 from both sides of the equation, we get:
x^3 = 12 - 7
x^3 = 5

Taking the cube root of both sides, we get:
x = ∛5

Therefore, the solution to the equation x^3 + 7 = 12 is x = ∛5.


In [54]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION:



In [65]:
functions = [
    {
        "name": "solver",
        "description": "Formulates and solves an equation",
        "parameters": {
            "type": "object",
            "properties": {
                "equation": {
                    "type": "string",
                    "description": "The algebraic expression of the equation",
                },
                "solution": {
                    "type": "string",
                    "description": "The solution to the equation",
                }
            },
            "required": ["equation", "solution"]
        }
    }
]

In [68]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Write out the following equation using algebraic symbols then solve it."),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model = 'gpt-4', temperature=0).bind(function_call={"name": "solver"}, functions=functions)

runnable = (
    {"equation_statement": RunnablePassthrough()} 
    | prompt 
    | model
)
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'solver', 'arguments': '{\n"equation": "x^3 + 7 = 12",\n"solution": "x = ∛5"\n}'}})

In [77]:
from langchain.chat_models import ChatOpenAI, ChatCohere

from unittest.mock import patch
from openai.error import RateLimitError

openai_llm = ChatOpenAI(max_retries=0)
cohere_llm = ChatCohere()
llm = openai_llm.with_fallbacks([cohere_llm])

with patch('openai.ChatCompletion.create', side_effect=RateLimitError()):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except:
        print("Hit error")

Hit error


In [79]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a nice assistant who always includes a compliment in your response"),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
with patch('openai.ChatCompletion.create', side_effect=RateLimitError()):
    try:
         print(chain.invoke({"animal": "kangaroo"}))
    except:
        print("Hit error")

content='Thank you for your kind words! I\'m trained to be polite and professional in all my responses.\n\nBut sorry, I didn\'t understand what you mean by "Why did the kangaroo cross the road?" Please clarify and I will gladly answer your query.'


In [84]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a nice assistant who always includes a compliment in your response"),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [85]:
# Now lets create a chain with the normal OpenAI model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm

In [86]:
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

"\n\nAnswer: The turtle crossed the road to get to the other side. That's quite a feat! I'm impressed by the turtle's determination and resourcefulness."

In [87]:
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | model

In [92]:
chain.invoke({"foo": "bae", "bar": "bobo"})

AIMessage(content='3 + 12 = 15')

In [94]:
from langchain.schema.runnable import RunnableConfig

def parse_or_fix(text:str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [97]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    res = RunnableLambda(parse_or_fix).invoke("{foo: bar, baz:, bas}", {"tags": "my-tag", "callbacks": [cb]})
    print(cb)
    print(res)

Tokens Used: 75
	Prompt Tokens: 60
	Completion Tokens: 15
Successful Requests: 1
Total Cost (USD): $0.00011999999999999999
{'foo': 'bar', 'baz': '', 'bas': None}


In [107]:
chain = PromptTemplate.from_template("""Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:""") | ChatCohere() | StrOutputParser()

In [115]:
chain.invoke({"question": "how do I call OpenAI, I wanted to ask where did all the cool people go when they left to start their own AI lab focused on safety. Something about constitutional AI and Dario Amodei"})

'LangChain'

In [116]:
langchain_chain = PromptTemplate.from_template("""You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatCohere()

anthropic_chain = PromptTemplate.from_template("""You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatCohere()

general_chain = PromptTemplate.from_template("""Respond to the following question:

Question: {question}
Answer:""") | ChatCohere()

In [117]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain
)

In [120]:
full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | branch

full_chain.invoke({"question": "what is 2 + 2?"})

AIMessage(content='The answer is 4.')

In [121]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [ ]:
from langchain.schema.runnable import RunnableLambda

full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | RunnableLambda(route)

In [123]:
full_chain.invoke({"question": "how do I simply use LangChain?"})

AIMessage(content="As Harrison Chase told me, LangChain is a no-code, drag-and-drop blockchain development platform focused on simplifying the implementation of blockchain use-cases. To use LangChain, users simply need to register on the platform and create a new project. Then, they can start building their blockchain application using the tools and templates available on the platform. Is there any specific aspect of LangChain you'd like to know more about?")

In [160]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          },
          "explanation": {
            "type": "string",
            "description": "Explanation for the joke"
          }
        },
        "required": ["setup", "punchline", "explanation"]
      },
    }, 
    {
      "name": "pun",
      "description": "A terrible pun",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the pun"
          },
          "stupid_twist": {
            "type": "string",
            "description": "The strange twist where the pun appears"
          }
        },
        "required": ["setup", "stupid_twist"]
      }
    }
  ]
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")

model = ChatOpenAI()
chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

res = chain.invoke({"foo": "bears"}, config={})


In [149]:
print(res.additional_kwargs['function_call'])

with get_openai_callback() as cb:
    fx = RunnableLambda(parse_or_fix).invoke(res.additional_kwargs['function_call'], {"tags": "my-tag", "callbacks": [cb]})
    print(cb)
    print(fx)

{'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears like fast food?",\n  "punchline": "Because they can\'t catch it!"\n}'}
Tokens Used: 252
	Prompt Tokens: 177
	Completion Tokens: 75
Successful Requests: 2
Total Cost (USD): $0.0004155
{'name': 'joke', 'arguments': {'setup': "Why don't bears like fast food?", 'punchline': "Because they can't catch it!"}}


In [150]:
chain = prompt | model | StrOutputParser()
chain.invoke({"foo": "bears"})


"Sure, here's a bear-related joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"

In [158]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser, JsonOutputFunctionsParser

chain = (
    prompt 
    | model.bind(function_call= {"name": "pun"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="stupid_twist")
)

chain.invoke({"foo": "fish"})

"Because they're afraid of the net!"

In [162]:
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

map_ = RunnableMap(foo = RunnablePassthrough())

chain = (
    map_
    | prompt
    | model.bind(function_call = {"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="explanation")
)

chain.invoke("fish")

"Fish don't like basketball because they are aquatic creatures and are not adapted for playing sports on land. The word 'net' has a double meaning here, referring to both the basketball net and a fishing net."